In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnull
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

# Creating Spark session
spark = SparkSession.builder.appName("SteamGamesSuccess").getOrCreate()

data = "games_march2025_full.csv"
df = spark.read.csv("games_march2025_full.csv", header=True, inferSchema=True)
df.show(truncate=False)



AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/c:/Users/jansu/Documents/BigData/BigDataTechnologies/games_march2025_full.csv.

In [ ]:
used_columns = [
    "appid", "name", "release_date", "genres", "tags", "developers", "publishers",
    "price", "discount", "recommendations", "positive", "negative", "peak_ccu", "dlc_count"
]

df1 = df.select(*used_columns)

df1.show(20)

+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+--------+---------------+--------+--------+--------+---------+
|  appid|                name|release_date|              genres|                tags|          developers|          publishers|price|discount|recommendations|positive|negative|peak_ccu|dlc_count|
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+--------+---------------+--------+--------+--------+---------+
|    730|    Counter-Strike 2|  21/08/2012|['Action', 'Free ...|{'FPS': 90857, 'S...|           ['Valve']|           ['Valve']|    0|       0|        4401572| 7480813| 1135108| 1212356|        1|
| 578080| PUBG: BATTLEGROUNDS|  21/12/2017|['Action', 'Adven...|{'Survival': 1483...|['PUBG Corporation']|   ['KRAFTON, Inc.']|    0|       0|        1732007| 1487960| 1024436|  616738|        0|
|    570|           

In [ ]:
df1 = df1.withColumnRenamed("discount", "discount_percentage")

In [ ]:
df1 = df1.withColumnRenamed("peak_ccu", "peak_playernum")

In [ ]:
df1.printSchema()

root
 |-- appid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- price: string (nullable = true)
 |-- discount_percentage: string (nullable = true)
 |-- recommendations: string (nullable = true)
 |-- positive: string (nullable = true)
 |-- negative: string (nullable = true)
 |-- peak_playernum: string (nullable = true)
 |-- dlc_count: string (nullable = true)



In [ ]:
print(f"Total Records: {df1.count()}")

df1.show()

Total Records: 94954
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------+--------+--------------+---------+
|  appid|                name|release_date|              genres|                tags|          developers|          publishers|price|discount_percentage|recommendations|positive|negative|peak_playernum|dlc_count|
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------+--------+--------------+---------+
|    730|    Counter-Strike 2|  21/08/2012|['Action', 'Free ...|{'FPS': 90857, 'S...|           ['Valve']|           ['Valve']|    0|                  0|        4401572| 7480813| 1135108|       1212356|        1|
| 578080| PUBG: BATTLEGROUNDS|  21/12/2017|['Action', 'Adven...|{'Survival': 1483...|['PUBG Corporation']|   ['KRAFTON, Inc.']|

# Data Preprocessing
- Handling Missing Values

In [ ]:
from pyspark.sql.functions import col, sum

null_counts = df1.select([sum(col(c).isNull().cast("int")).alias(c) for c in df1.columns])
null_counts.show()

+-----+----+------------+------+----+----------+----------+-----+-------------------+---------------+--------+--------+--------------+---------+
|appid|name|release_date|genres|tags|developers|publishers|price|discount_percentage|recommendations|positive|negative|peak_playernum|dlc_count|
+-----+----+------------+------+----+----------+----------+-----+-------------------+---------------+--------+--------+--------------+---------+
|    0|   2|           0|     5|  10|         9|         9|    0|                 11|             10|     110|      75|             9|        0|
+-----+----+------------+------+----+----------+----------+-----+-------------------+---------------+--------+--------+--------------+---------+



In [ ]:
# We dropped null name rows, because a game without a name is invalid.
# We dropped null tag rows, because there were quite a few of them and we do not get much out of them.
# We dropped null peak_playernum rows, because if it has had no peak it means it it does not have a playerbase, so we do not need it.
df_cleaned = df1.dropna(subset=["name", "tags", "peak_playernum"])

# We changed null genres into "unknown", because there were only two of them. It won't impact the analyzis much.
df_filled = df_cleaned.fillna({"genres": "unknown", "developers": "unknown", "publishers": "unknown", "discount_percentage": "0", "recommendations": "0"
                               ,"positive": "0", "negative": "0"})




In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, regexp_replace, trim, when, round

In [ ]:
df = df_filled.withColumn(
    "positive_clean",
    when(
        trim(col("positive")).rlike("^\d+$"),
        trim(col("positive")).cast("double")
    ).otherwise(0)
).withColumn(
    "negative_clean",
    when(
        trim(col("negative")).rlike("^\d+$"),
        trim(col("negative")).cast("double")
    ).otherwise(0)
).withColumn(
    "all_reviews",
    round(col("positive_clean") + col("negative_clean"), 2)
)


In [ ]:
#creating the percentage of reviews columns
df_with_percentages = df.withColumn(
    "positive_percentage", 
    (F.col("positive") / F.col("all_reviews") * 100).cast("double")
).withColumn(
    "negative_percentage", 
    (F.col("negative") / F.col("all_reviews") * 100).cast("double")
)

df_with_percentages = df_with_percentages.withColumn(
    "positive_percentage", 
    F.round(F.col("positive_percentage"), 2)
).withColumn(
    "negative_percentage", 
    F.round(F.col("negative_percentage"), 2)
)

df_with_percentages.show()


+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------+--------+--------------+---------+--------------+--------------+-----------+-------------------+-------------------+
|  appid|                name|release_date|              genres|                tags|          developers|          publishers|price|discount_percentage|recommendations|positive|negative|peak_playernum|dlc_count|positive_clean|negative_clean|all_reviews|positive_percentage|negative_percentage|
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------+--------+--------------+---------+--------------+--------------+-----------+-------------------+-------------------+
|    730|    Counter-Strike 2|  21/08/2012|['Action', 'Free ...|{'FPS': 90857, 'S...|           ['Valve']|         

In [ ]:
df = df_with_percentages.fillna({"positive_percentage": "0", "negative_percentage": "0"})

In [ ]:
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+-----+----+------------+------+----+----------+----------+-----+-------------------+---------------+--------+--------+--------------+---------+--------------+--------------+-----------+-------------------+-------------------+
|appid|name|release_date|genres|tags|developers|publishers|price|discount_percentage|recommendations|positive|negative|peak_playernum|dlc_count|positive_clean|negative_clean|all_reviews|positive_percentage|negative_percentage|
+-----+----+------------+------+----+----------+----------+-----+-------------------+---------------+--------+--------+--------------+---------+--------------+--------------+-----------+-------------------+-------------------+
|    0|   0|           0|     0|   0|         0|         0|    0|                  0|              0|       0|       0|             0|        0|             0|             0|          0|                  0|                  0|
+-----+----+------------+------+----+----------+----------+-----+-------------------+-------

In [ ]:
df.show()

+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------+--------+--------------+---------+--------------+--------------+-----------+-------------------+-------------------+
|  appid|                name|release_date|              genres|                tags|          developers|          publishers|price|discount_percentage|recommendations|positive|negative|peak_playernum|dlc_count|positive_clean|negative_clean|all_reviews|positive_percentage|negative_percentage|
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------+--------+--------------+---------+--------------+--------------+-----------+-------------------+-------------------+
|    730|    Counter-Strike 2|  21/08/2012|['Action', 'Free ...|{'FPS': 90857, 'S...|           ['Valve']|         

**Duplicates**

In [ ]:
id_duplicates = df.groupBy("appid").count().filter("count > 1")
name_duplicates = df.groupBy("name").count().filter("count > 1")

id_duplicates.show()
name_duplicates.show()

+--------------------+-----+
|               appid|count|
+--------------------+-----+
|nd deciding the s...|    2|
|   ahead of schedule|    2|
+--------------------+-----+

+--------------------+-----+
|                name|count|
+--------------------+-----+
|      Eternal Return|    2|
|               Nomad|    2|
|Loading Screen Si...|    2|
|The Lord of the R...|    2|
|             Journey|    2|
|Call of Duty®: Bl...|    2|
|        Blood Strike|    2|
|Call of Duty®: Bl...|    2|
|    EA SPORTS FC™ 24|    4|
|              ISLAND|    2|
|Romance of the Th...|    2|
|            Downfall|    2|
|     torpedo strikes|    2|
|                Home|    2|
|Ys I & II Chronic...|    2|
| Monday Night Combat|    2|
|               Chasm|    2|
|       Second Chance|    2|
|      Hero's Journey|    3|
|          The Bunker|    3|
+--------------------+-----+
only showing top 20 rows



In [ ]:
df_no_duplicates = df.dropDuplicates(["name"])

name_duplicates = df_no_duplicates.groupBy("name").count().filter("count > 1")
name_duplicates.show()

+----+-----+
|name|count|
+----+-----+
+----+-----+



In [ ]:
used_columns = [
    "appid", "name", "release_date", "genres", "tags", "developers", "publishers",
    "price", "discount_percentage", "recommendations", "peak_playernum", "dlc_count", "all_reviews", "positive_percentage", "negative_percentage"
]

df_final = df_no_duplicates.select(*used_columns)

In [ ]:
df_final.show(10)

+-------+------------------------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+
|  appid|                                name|release_date|              genres|                tags|          developers|          publishers|price|discount_percentage|recommendations|peak_playernum|dlc_count|all_reviews|positive_percentage|negative_percentage|
+-------+------------------------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+
|3066390|"軍艦島探訪記　ある写真家の記録　...|  01/08/2024|      ['Simulation']|{'Simulation': 53...|         ['XYimage']|         ['XYimage']|19.99|                  0|              0|             0|        0|        4.0|     

# Spark optimization


In [ ]:
# Cache the DataFrame if it will be used multiple times
df_final.cache()


DataFrame[appid: string, name: string, release_date: string, genres: string, tags: string, developers: string, publishers: string, price: string, discount_percentage: string, recommendations: string, peak_playernum: string, dlc_count: string, all_reviews: double, positive_percentage: double, negative_percentage: double]

In [ ]:
# Show the execution plan to check optimizations
df_final.explain(True)


== Parsed Logical Plan ==
'Project ['appid, 'name, 'release_date, 'genres, 'tags, 'developers, 'publishers, 'price, 'discount_percentage, 'recommendations, 'peak_playernum, 'dlc_count, 'all_reviews, 'positive_percentage, 'negative_percentage]
+- Deduplicate [name#18]
   +- Project [appid#17, name#18, release_date#19, genres#6539, tags#58, developers#6540, publishers#6541, price#21, discount_percentage#6542, recommendations#6543, positive#6544, negative#6545, peak_playernum#6237, dlc_count#22, positive_clean#7504, negative_clean#7520, all_reviews#7537, coalesce(nanvl(positive_percentage#8089, cast(null as double)), cast(0 as double)) AS positive_percentage#8239, coalesce(nanvl(negative_percentage#8109, cast(null as double)), cast(0 as double)) AS negative_percentage#8240]
      +- Project [appid#17, name#18, release_date#19, genres#6539, tags#58, developers#6540, publishers#6541, price#21, discount_percentage#6542, recommendations#6543, positive#6544, negative#6545, peak_playernum#6237,

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

# Define a UDF to classify games based on positive_percentage and recommendations
def classify_success(positive_percentage, recommendations):
    try:
        recommendations_int = int(recommendations)  # Convert recommendations to an integer
    except ValueError:
        recommendations_int = 0  # Default to 0 if conversion fails

    # Define conditions for classification
    if positive_percentage > 80 and recommendations_int > 1000:
        return "Successful"
    elif positive_percentage > 50 and recommendations_int > 100:
        return "Moderate"
    else:
        return "Unsuccessful"

# Register the UDF
classify_success_udf = udf(classify_success, StringType())

# Apply the UDF to add a new column 'success_category' based on the classification
games_with_success_classification = df_final.withColumn(
    "success_category", 
    classify_success_udf(col("positive_percentage"), col("recommendations"))
)

# Show the resulting DataFrame with the new 'success_category' column
games_with_success_classification.show()


+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|  appid|                name|release_date|              genres|                tags|          developers|          publishers| price|discount_percentage|recommendations|peak_playernum|dlc_count|all_reviews|positive_percentage|negative_percentage|success_category|
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|1963980|                 ...|  04/05/2022|['Action', 'Adven...|{'Adventure': 67,...|      ['GDE ANIMES']|      ['GDE ANIMES']|  1.99|                  0|              0|             0|        0|        8.

In [ ]:
games_with_success_classification.createOrReplaceTempView("games")

In [ ]:
spark.sql("SELECT * FROM games").show(10)

+-------+------------------------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|  appid|                                name|release_date|              genres|                tags|          developers|          publishers|price|discount_percentage|recommendations|peak_playernum|dlc_count|all_reviews|positive_percentage|negative_percentage|success_category|
+-------+------------------------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|3066390|"軍艦島探訪記　ある写真家の記録　...|  01/08/2024|      ['Simulation']|{'Simulation': 53...|         ['XYimage']|         ['XYimage']|19.99|                  0|       

In [ ]:
spark.sql("SELECT * FROM games WHERE success_category NOT LIKE 'Unsuccessful'").show()

+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|  appid|                name|release_date|              genres|                tags|          developers|          publishers|price|discount_percentage|recommendations|peak_playernum|dlc_count|all_reviews|positive_percentage|negative_percentage|success_category|
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|1864790|          1BITDRAGON|  18/03/2022|['Audio Productio...|{'Audio Productio...|      ['1BITDRAGON']|      ['1BITDRAGON']|29.99|                  0|            205|             1|        0|      226.0|  

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#Filter out rows with 'Unsuccessful' success_category
filtered_df = games_with_success_classification.filter(F.col("success_category") != "Unsuccessful")

+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|  appid|                name|release_date|              genres|                tags|          developers|          publishers|price|discount_percentage|recommendations|peak_playernum|dlc_count|all_reviews|positive_percentage|negative_percentage|success_category|
+-------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----+-------------------+---------------+--------------+---------+-----------+-------------------+-------------------+----------------+
|1864790|          1BITDRAGON|  18/03/2022|['Audio Productio...|{'Audio Productio...|      ['1BITDRAGON']|      ['1BITDRAGON']|29.99|                  0|            205|             1|        0|      226.0|  

## Comparing the overall success of the games in 2025

In [ ]:
#Categorize prices
categorized_df = filtered_df.withColumn(
    "price_category",
    F.when(F.col("price") == 0, "Free")
     .when(F.col("price") < 10, "Budget")
     .when(F.col("price") < 30, "Mid-range")
     .otherwise("Premium")
)

In [ ]:
success_order = F.when(F.col("success_category") == "Successful", 3) \
                 .when(F.col("success_category") == "Moderate", 2) \
                 .when(F.col("success_category") == "Unsuccessful", 1) \
                 .otherwise(0)


In [ ]:
# numeric rank column
categorized_df = categorized_df.withColumn("success_rank", success_order)

# Window ordered by success_rank descending
price_window = Window.partitionBy("price_category").orderBy(F.desc("success_rank"))

# Apply ranking
price_analysis = categorized_df.withColumn("rank_in_category", F.rank().over(price_window))
price_analysis.select("appid","name", "price", "price_category", "success_category", "success_rank", "rank_in_category").show(truncate=False)


+----------------------------------------------+-----+--------------+----------------+------------+----------------+
|name                                          |price|price_category|success_category|success_rank|rank_in_category|
+----------------------------------------------+-----+--------------+----------------+------------+----------------+
|100% Orange Juice                             |6.99 |Budget        |Successful      |3           |1               |
|12 Labours of Hercules                        |2.99 |Budget        |Successful      |3           |1               |
|60 Seconds! Reatomized                        |9.99 |Budget        |Successful      |3           |1               |
|64                                            |2.99 |Budget        |Successful      |3           |1               |
|9-nine-:Episode 4                             |9.99 |Budget        |Successful      |3           |1               |
|A Kiss For The Petals - Remembering How We Met|7.99 |Budget    

In [ ]:
success_categories = ["Successful", "Moderate", "Unsuccessful"]

for category in success_categories:
    count = games_with_success_classification.filter(
        col("success_category") == category
    ).count()

    print(f"{category}: {count} games")

In [ ]:
best_games = price_analysis.filter(F.col("rank_in_category") != "3")
best_games.select("appid", "name", "price", "price_category", "success_category", "success_rank", "rank_in_category").show(truncate=False)

In [ ]:
best_games.count()

In [ ]:
from pyspark.sql.functions import explode, split, regexp_replace, col

# Step 1: Filter Successful and Moderate Games
best_games = price_analysis.filter(F.col("rank_in_category") != "3")

# Step 2: Exploding toilets
df_exploded_tags = best_games.withColumn(
    "tag", 
    explode(split(regexp_replace("tags", "[\\[\\]' ]", ""), ","))
)

# Step 3: Clean the tag by removing numbers after the colon
df_exploded_tags_clean = df_exploded_tags.withColumn(
    "clean_tag", 
    split(col("tag"), ":").getItem(0)  # Keeps only the first part (before the colon)
)

# Step 4: Count tags for successful games
df_successful = df_exploded_tags_clean.filter(col("success_category") == "Successful") \
    .groupBy("clean_tag").count().withColumnRenamed("count", "successful_count")

# Step 5: Count tags for moderate games
df_moderate = df_exploded_tags_clean.filter(col("success_category") == "Moderate") \
    .groupBy("clean_tag").count().withColumnRenamed("count", "moderate_count")

# Step 6: Perform a join on clean_tag to compare counts between successful and moderate games
df_joined = df_successful.join(
    df_moderate,
    on="clean_tag",  # Joining on the tag name
    how="outer"  # Outer join to include tags from both categories
)

# Step 7: Order the results based on the successful_count (highest successful_count first)
df_joined_ordered = df_joined.orderBy("successful_count", ascending=False)  # Order by successful_count, highest first

# Step 8: Show the results ordered by the successful count of tags
df_joined_ordered.show(10, truncate=False)